In [1]:
############ IMPORTS ####################
import sys
sys.path.append("../")
import os
from os import path
import numpy as np
import random
import torch
import torch.utils.data as dataf
import torch.nn as nn
import matplotlib.pyplot as plt
from scipy import io as sio
from sklearn.decomposition import PCA
from torch.nn.parameter import Parameter
import torchvision.transforms.functional as TF
import torch.nn.functional as F
import time
import math
from operator import truediv
from sklearn.metrics import confusion_matrix, accuracy_score, cohen_kappa_score
from torchsummary import summary
import record

In [10]:
############ CONFIGS ####################
os.environ["CUDA_VISIBLE_DEVICES"]="0"
datasetNames = ["Trento"]
testSizeNumber = 2500
batchsize = 64
EPOCH = 200
LR = 0.001
patchsize = 11
NUM_ITERATIONS = 3
FM = 16

In [3]:
class Conv3d_cd(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=3, stride=1,
                 padding=1, dilation=1, groups=1, bias=False, theta=0.7):

        super(Conv3d_cd, self).__init__() 
        self.conv = nn.Conv3d(in_channels, out_channels, kernel_size=kernel_size, stride=stride, padding=padding, dilation=dilation, groups=groups, bias=bias)
        self.theta = theta

    def forward(self, x):
        out_normal = self.conv(x)


        if math.fabs(self.theta - 0.0) < 1e-8:
            return out_normal 
        else:
            #pdb.set_trace()
            [C_out,C_in, kernel_size,kernel_size,depth] = self.conv.weight.shape
            kernel_diff = self.conv.weight.sum(2).sum(2)
            kernel_diff = kernel_diff[:, :, None, None]
            kernel_diff = kernel_diff.repeat(1,1,3,3,1)
            out_diff = F.conv3d(input=x, weight=kernel_diff, bias=self.conv.bias, stride=self.conv.stride, padding=self.conv.padding, groups=self.conv.groups)

            return out_normal - self.theta * out_diff

In [4]:
class CNN(nn.Module):
    def __init__(self, FM, Classes, patchsize, NC):
        super(CNN, self).__init__()
        self.conv1 = nn.Sequential(
            Conv3d_cd(
                in_channels = 1,
                out_channels = FM,
                kernel_size = (3, 3, 7),
                stride = 1,
                padding = (0,0,0)
            ),
            nn.BatchNorm3d(FM),
            nn.ReLU(),
            nn.MaxPool3d(kernel_size=(1,1,2)),
#             nn.Dropout(0.5),
        )
        self.final_bands = (NC - 6) // 2
        
        self.conv2 = nn.Sequential(
            Conv3d_cd(FM, FM*2, (3, 3, 7 ), 1, (0,0,0)),
            nn.BatchNorm3d(FM*2),
            nn.ReLU(),
            nn.MaxPool3d(kernel_size=(1,1,2))

        )
        self.final_bands = (self.final_bands - 6) // 2
        self.conv3 = nn.Sequential(
            Conv3d_cd(FM*2, FM*4, (3, 3, 7), 1, (0,0,0)),
            nn.BatchNorm3d(FM*4),
            nn.ReLU(),
            nn.MaxPool3d(kernel_size=(1,1,2))

        )
        self.final_bands = (self.final_bands - 6) // 2
        
        self.final_patch_size = patchsize - 6
        
        
        
        self.out1 =  nn.Linear(self.final_patch_size * self.final_patch_size * FM * 4 * self.final_bands, Classes)
#         self.out1 =  nn.Linear(19200, Classes)
        
    def forward(self, x1):
        x1 = x1.unsqueeze(1)
        x1 = self.conv1(x1)
        x1 = self.conv2(x1)
        x1 = self.conv3(x1)
        x1 = x1.reshape(x1.shape[0], -1)  # flatten the output of conv2 to (batch_size, 32 * 7 * 7)
        out1 = self.out1(x1)
        return out1

In [5]:
def AA_andEachClassAccuracy(confusion_matrix):
    counter = confusion_matrix.shape[0]
    list_diag = np.diag(confusion_matrix)
    list_raw_sum = np.sum(confusion_matrix, axis=1)
    each_acc = np.nan_to_num(truediv(list_diag, list_raw_sum))
    average_acc = np.mean(each_acc)
    return each_acc, average_acc

def reports (xtest,ytest,name):
    pred_y = np.empty((len(ytest)), dtype=np.float32)
    number = len(ytest) // testSizeNumber
    for i in range(number):
        temp = xtest[i * testSizeNumber:(i + 1) * testSizeNumber, :, :, :]
        temp = temp.cuda()

        temp2 = cnn(temp)
        temp3 = torch.max(temp2, 1)[1].squeeze()
        pred_y[i * testSizeNumber:(i + 1) * testSizeNumber] = temp3.cpu()
        del temp, temp2, temp3

    if (i + 1) * testSizeNumber < len(ytest):
        temp = xtest[(i + 1) * testSizeNumber:len(ytest), :, :, :]
        temp = temp.cuda()


        temp2 = cnn(temp)
        temp3 = torch.max(temp2, 1)[1].squeeze()
        pred_y[(i + 1) * testSizeNumber:len(ytest)] = temp3.cpu()
        del temp, temp2, temp3

    pred_y = torch.from_numpy(pred_y).long()

    
    oa = accuracy_score(ytest, pred_y)
    confusion = confusion_matrix(ytest, pred_y)
    each_acc, aa = AA_andEachClassAccuracy(confusion)
    kappa = cohen_kappa_score(ytest, pred_y)

    return confusion, oa*100, each_acc*100, aa*100, kappa*100


In [6]:
def Train_And_Validate(EPOCH,train_loader,loss_func,optimizer,TestLabel,cnn):
    BestAcc = 0
    torch.cuda.synchronize()
    start = time.time()
    for epoch in range(EPOCH):
        for step, (b_x1,b_y) in enumerate(train_loader):
            # move train data to GPU
            b_x1 = b_x1.cuda()
            b_y = b_y.cuda()

            out1 = cnn(b_x1)
            loss = loss_func(out1, b_y)

            optimizer.zero_grad()  # clear gradients for this training step
            loss.backward()  # backpropagation, compute gradients
            optimizer.step()  # apply gradients

            if step == len(train_loader) - 1:
                cnn.eval()
                pred_y = np.empty((len(TestLabel)), dtype='float32')
                number = len(TestLabel) // testSizeNumber
                for i in range(number):
                    temp = TestPatch[i * testSizeNumber:(i + 1) * testSizeNumber, :, :, :]
                    temp = temp.cuda()

                    temp2 = cnn(temp)
                    temp3 = torch.max(temp2, 1)[1].squeeze()
                    pred_y[i * testSizeNumber:(i + 1) * testSizeNumber] = temp3.cpu()
                    del temp, temp2, temp3


                if (i + 1) * testSizeNumber < len(TestLabel):
                    temp = TestPatch[(i + 1) * testSizeNumber:len(TestLabel), :, :, :]
                    temp = temp.cuda()

                    temp2 = cnn(temp)
                    temp3 = torch.max(temp2, 1)[1].squeeze()
                    pred_y[(i + 1) * testSizeNumber:len(TestLabel)] = temp3.cpu()
                    del temp, temp2, temp3

                pred_y = torch.from_numpy(pred_y).long()
                accuracy = torch.sum(pred_y == TestLabel).type(torch.FloatTensor) / TestLabel.size(0)

                print('Epoch: ', epoch, '| train loss: %.4f' % loss.data.cpu().numpy(), '| test accuracy: %.2f' % accuracy)

                # save the parameters in network
                if accuracy > BestAcc:
                    BestAcc = accuracy
                    torch.save(cnn.state_dict(), datasetName+'/net_params_checkpoint_G2C_Conv3D.pkl')
                cnn.train()
    torch.cuda.synchronize()
    end = time.time()
    print("Time taken to train = ",end - start, "s")
    Train_time = end - start
    return Train_time

In [11]:
#################### Disjoint Dataset Training ####################

def set_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    

for datasetName in datasetNames:

    print("Current dataset = ",datasetName)

    try:
        os.makedirs(datasetName)
    except FileExistsError:
        pass

    # Train data
    HSI = sio.loadmat('./../'+datasetName+'11x11/HSI_Tr.mat')
    TrainPatch = HSI['Data']
    TrainPatch = TrainPatch.astype(np.float32)
    NC = TrainPatch.shape[3] # NC is number of bands

    label = sio.loadmat('./../'+datasetName+'11x11/TrLabel.mat')
    TrLabel = label['Data']

    # Test data
    HSI = sio.loadmat('./../'+datasetName+'11x11/HSI_Te.mat')
    TestPatch = HSI['Data']
    TestPatch = TestPatch.astype(np.float32)

    label = sio.loadmat('./../'+datasetName+'11x11/TeLabel.mat')
    TsLabel = label['Data']


    TrainPatch = torch.from_numpy(TrainPatch)
#     TrainPatch = TrainPatch.permute(0,3,1,2)
    TrainLabel = torch.from_numpy(TrLabel)-1
    TrainLabel = TrainLabel.reshape(-1).long()


    TestPatch = torch.from_numpy(TestPatch)
#     TestPatch = TestPatch.permute(0,3,1,2)
    TestLabel = torch.from_numpy(TsLabel)-1
    TestLabel = TestLabel.reshape(-1).long()

    Classes = len(np.unique(TrainLabel))

    dataset = dataf.TensorDataset(TrainPatch, TrainLabel)
    train_loader = dataf.DataLoader(dataset, batch_size=batchsize, shuffle=True)

    print("Train data shape = ", TrainPatch.shape)
    print("Train label shape = ", TrainLabel.shape)
    print("Test data shape = ", TestPatch.shape)
    print("Test label shape = ", TestLabel.shape)
    print("Classes = ", Classes)

    KAPPA = []
    OA = []
    AA = []
    ELEMENT_ACC = np.zeros((NUM_ITERATIONS, Classes))
    set_seed(42)
    for iterNum in range(NUM_ITERATIONS):    
        cnn = CNN(FM, Classes, patchsize,NC)
        cnn = cnn.cuda()
        summary(cnn, (patchsize, patchsize, NC))
        optimizer = torch.optim.Adam(cnn.parameters(), lr=LR)
        loss_func = nn.CrossEntropyLoss()  # the target label is not one-hotted
        train_time = Train_And_Validate(EPOCH,train_loader,loss_func,optimizer,TestLabel,cnn)

        cnn.load_state_dict(torch.load(datasetName+'/net_params_checkpoint_G2C_Conv3D.pkl'))
        cnn.eval()


        confusion, oa, each_acc, aa, kappa = reports(TestPatch,TestLabel,datasetName)
        KAPPA.append(kappa)
        OA.append(oa)
        AA.append(aa)
        ELEMENT_ACC[iterNum, :] = each_acc
        torch.save(cnn, datasetName+'/best_model_G2C-Conv3D-HSI_PatchSizetest'+str(patchsize)+'_Iter'+str(iterNum)+'_'+ datasetName +'.pt')
    print("--------" + datasetName + " Training Finished-----------")
    record.record_output(OA, AA, KAPPA, ELEMENT_ACC,'./' + datasetName +'/G2C-Conv3D-HSI_PatchSizetest'+str(patchsize)+'_Report_' + datasetName +'.txt')
                        
                        
        

Current dataset =  Trento
Train data shape =  torch.Size([819, 11, 11, 63])
Train label shape =  torch.Size([819])
Test data shape =  torch.Size([29395, 11, 11, 63])
Test label shape =  torch.Size([29395])
Classes =  6
------------------------------------------------------------------------------------------
Layer (type:depth-idx)                   Output Shape              Param #
├─Sequential: 1-1                        [-1, 16, 9, 9, 28]        --
|    └─Conv3d_cd: 2-1                    [-1, 16, 9, 9, 57]        --
|    |    └─Conv3d: 3-1                  [-1, 16, 9, 9, 57]        1,008
|    └─BatchNorm3d: 2-2                  [-1, 16, 9, 9, 57]        32
|    └─ReLU: 2-3                         [-1, 16, 9, 9, 57]        --
|    └─MaxPool3d: 2-4                    [-1, 16, 9, 9, 28]        --
├─Sequential: 1-2                        [-1, 32, 7, 7, 11]        --
|    └─Conv3d_cd: 2-5                    [-1, 32, 7, 7, 22]        --
|    |    └─Conv3d: 3-2                  [-1, 32, 7,

/home/users/jgecvision/.conda/envs/purb37/lib/python3.7/site-packages/torch/serialization.py:402: UserWarning: Couldn't retrieve source code for container of type CNN. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/home/users/jgecvision/.conda/envs/purb37/lib/python3.7/site-packages/torch/serialization.py:402: UserWarning: Couldn't retrieve source code for container of type Conv3d_cd. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


------------------------------------------------------------------------------------------
Layer (type:depth-idx)                   Output Shape              Param #
├─Sequential: 1-1                        [-1, 16, 9, 9, 28]        --
|    └─Conv3d_cd: 2-1                    [-1, 16, 9, 9, 57]        --
|    |    └─Conv3d: 3-1                  [-1, 16, 9, 9, 57]        1,008
|    └─BatchNorm3d: 2-2                  [-1, 16, 9, 9, 57]        32
|    └─ReLU: 2-3                         [-1, 16, 9, 9, 57]        --
|    └─MaxPool3d: 2-4                    [-1, 16, 9, 9, 28]        --
├─Sequential: 1-2                        [-1, 32, 7, 7, 11]        --
|    └─Conv3d_cd: 2-5                    [-1, 32, 7, 7, 22]        --
|    |    └─Conv3d: 3-2                  [-1, 32, 7, 7, 22]        32,256
|    └─BatchNorm3d: 2-6                  [-1, 32, 7, 7, 22]        64
|    └─ReLU: 2-7                         [-1, 32, 7, 7, 22]        --
|    └─MaxPool3d: 2-8                    [-1, 32, 7, 7, 1

/home/users/jgecvision/.conda/envs/purb37/lib/python3.7/site-packages/torch/serialization.py:402: UserWarning: Couldn't retrieve source code for container of type CNN. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/home/users/jgecvision/.conda/envs/purb37/lib/python3.7/site-packages/torch/serialization.py:402: UserWarning: Couldn't retrieve source code for container of type Conv3d_cd. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


------------------------------------------------------------------------------------------
Layer (type:depth-idx)                   Output Shape              Param #
├─Sequential: 1-1                        [-1, 16, 9, 9, 28]        --
|    └─Conv3d_cd: 2-1                    [-1, 16, 9, 9, 57]        --
|    |    └─Conv3d: 3-1                  [-1, 16, 9, 9, 57]        1,008
|    └─BatchNorm3d: 2-2                  [-1, 16, 9, 9, 57]        32
|    └─ReLU: 2-3                         [-1, 16, 9, 9, 57]        --
|    └─MaxPool3d: 2-4                    [-1, 16, 9, 9, 28]        --
├─Sequential: 1-2                        [-1, 32, 7, 7, 11]        --
|    └─Conv3d_cd: 2-5                    [-1, 32, 7, 7, 22]        --
|    |    └─Conv3d: 3-2                  [-1, 32, 7, 7, 22]        32,256
|    └─BatchNorm3d: 2-6                  [-1, 32, 7, 7, 22]        64
|    └─ReLU: 2-7                         [-1, 32, 7, 7, 22]        --
|    └─MaxPool3d: 2-8                    [-1, 32, 7, 7, 1

/home/users/jgecvision/.conda/envs/purb37/lib/python3.7/site-packages/torch/serialization.py:402: UserWarning: Couldn't retrieve source code for container of type CNN. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/home/users/jgecvision/.conda/envs/purb37/lib/python3.7/site-packages/torch/serialization.py:402: UserWarning: Couldn't retrieve source code for container of type Conv3d_cd. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
